In [2]:
from cmlreaders import CMLReader, get_data_index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy
from scipy import stats

In [4]:
BAI_high = 15
BAI_low = 8
BDI_high = 16
BDI_low = 10

sel_sub = pd.DataFrame([])
master = pd.read_excel(r'./All_neuropsych.xlsx',engine="openpyxl") #read master spreadsheet of all RAM subjects demo/neuropsych data

sub_mat=list(np.zeros(len(master))*np.nan)
df = get_data_index('r1')
# df_select = pd.DataFrame([])
for sub in df['subject'].unique():
    num = int(sub[2:5])
    for i in master.index:
        sel = master.iloc[i]['Subject Number']
        if sel == num:
            sub_mat[i]=sub
# df_select[0:50]
master['subject']=sub_mat



,Subject Number,Event Name,Study site,Gender,Years of Education,Age of seizure onset,Age at Implant (calculated),Phase II Implant Date,Phase III Implant Date,Traumatic Brain Injury (TBI) Notes,...,FSIQ,GAI,VCI,PRI,Pre-operative Report,Prior Neurosurgical Procedures (can select more than one),Prior Surgery: Resection Detail,Prior Surgery: Callosotomy Detail,Prior Surgery: Laser Ablation Detail,subject
0,1,Subject Consent & Surgery,University of Pennsylvania,Female,13.0,20.0,48.105026,2014-10-08,NaT,NaN,...,72,76.0,81,75,REP001_neurospych.pdf,NaN,NaN,NaN,NaN,R1001P
1,2,Subject Consent & Surgery,University of Pennsylvania,Female,16.0,34.0,49.383743,2014-11-12,NaT,NaN,...,86,84.0,89,84,R1002P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1002P
2,3,Subject Consent & Surgery,University of Pennsylvania,Female,13.0,37.0,39.390382,2014-11-13,NaT,motor vehicle accident due to loss of consciou...,...,102,NaN,107,100,R1003P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1003P
3,4,Subject Consent & Surgery,Dartmouth University,Female,12.0,6.0,51.918931,2014-11-06,NaT,"In early 90s, motor vehicle accident resulting...",...,72,NaN,78,82,R1004D Neuropsych.pdf,NaN,NaN,NaN,NaN,R1004D
4,5,Subject Consent & Surgery,University of Pennsylvania,Female,NaN,NaN,20.304432,2015-01-14,NaT,NaN,...,80,89.0,98,82,R1005P_neuropsych.pdf,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,592,Subject Consent & Surgery,Emory University,Female,15.0,9.0,21.843022,2021-11-10,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
592,593,Subject Consent & Surgery,Dartmouth University,Female,20.0,2.0,37.408138,2021-11-29,NaT,NaN,...,100,104.0,100,109,R1593D_Neuropsych_15Mar2021.pdf,VNS implant,NaN,NaN,NaN,NaN
593,594,Subject Consent & Surgery,Emory University,Male,16.0,36.0,38.276054,2021-11-17,NaT,NaN,...,NaN,107.0,114,100,R1594E-Neuropsychologic Report_redacted.pdf,NaN,NaN,NaN,NaN,R1594E
594,595,Subject Consent & Surgery,UT Southwestern,Female,NaN,NaN,20.950464,2021-11-23,NaT,NaN,...,70,74.0,5,6,R1595T.docx,NaN,NaN,NaN,NaN,NaN


In [6]:
trial_df = pd.DataFrame([])
list_df = pd.DataFrame([])

experiments = ['FR1', 'catFR1']

for sub in master['subject']:
#   try:
    for exp in experiments:
        localizations=df[(df['subject']==sub) & (df['experiment']==exp)]['localization'].unique()        

        for loc in localizations:
            montages = df[(df['subject']==sub) & (df['experiment']==exp) & (df['localization']==loc)]['montage'].unique()            

            for mont in montages: # iterate over montages            
                data = df[np.logical_and(df["subject"] == sub, df['experiment']==exp)]
                sessions=data[np.logical_and(data['montage']==mont,data['localization']==loc)]['session'].unique()

                for sess in sessions: 
                    reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)   #reader for loading CML data
                    evs = reader.load('task_events')
                    word_evs = evs[evs['type']=='REC_WORD']

                    #exclude practice lists 
                    lists = len(word_evs['list'][word_evs['list']>0].unique()) 
                    corr_recs = len(word_evs[word_evs['intrusion']==0])
                    ELIs = len(word_evs[word_evs['intrusion']==-1])
                    PLIs = len(word_evs[word_evs['intrusion']>0])
                    total_ints = ELIs + PLIs
                    total_trials = corr_recs + total_ints

                    trials = [] #make list of event numbers of analysis
                    for list_num in word_evs['list'][word_evs['list']>0].unique():
                        list_events=word_evs[word_evs['list']==list_num]
                        if len(list_events)>0: 
                            for i in range(0,len(list_events)):
                                trials.append(list_events.index[i])
#                         PLI = len(list_events[list_events['intrusion']>0])
#                         ELI = len(list_events[list_events['intrusion']==-1])
#                         correc = len(list_events[list_events['intrusion']==0])
#                         list_df = list_df.append({ #create dataframe of events for analysis grouped by list ##should I filter out lists with no events?
#                             'subject':sub,
#                             'session': sess,
#                             'experiment': exp,
#                             'list': list_num,
#                             'number of correct recalls': correc,
#                             'number of extra-list intrusions': ELI,
#                             'number of prior-list intrusions': PLI,
#                             'total number of intrusions': ELI+PLI,
#                             'BAI':BAI,
#                             'BDI':BDI
#                             },ignore_index=True)
                    row = master[master['subject']==sub] #get row of BAI/BDI scores for subject
                    BAI = row['BAI'].item()
                    BDI = row['BDI'].item()
                    trial_df = trial_df.append({ #create dataframe of events for analysis grouped by session
                        'subject':sub,
                        'session': sess,
                        'montage': mont,
                        'localization': loc,
                        'experiment': exp,
                        'number of lists': lists,
                        'number of correct recalls': corr_recs,
                        'number of extra-list intrusions': ELIs,
                        'number of prior-list intrusions': PLIs,
                        'total number of intrusions': total_ints,
                        'total number of trials left': total_trials,
                        'trials for analysis': trials
                        },ignore_index=True) 
#     except:
#         print('Failed for subject:', sub)
trial_df[:100]

,experiment,localization,montage,number of correct recalls,number of extra-list intrusions,number of lists,number of prior-list intrusions,session,subject,total number of intrusions,total number of trials left,trials for analysis
0,FR1,0.0,0.0,62.0,21.0,22.0,25.0,0.0,R1001P,46.0,108.0,"[63, 64, 66, 67, 88, 89, 90, 91, 92, 93, 95, 9..."
1,FR1,0.0,0.0,63.0,19.0,23.0,17.0,1.0,R1001P,36.0,99.0,"[38, 39, 40, 41, 62, 63, 64, 65, 66, 67, 68, 8..."
2,FR1,0.0,0.0,117.0,3.0,25.0,8.0,0.0,R1002P,11.0,128.0,"[38, 39, 40, 41, 67, 68, 69, 70, 71, 74, 95, 9..."
3,FR1,0.0,0.0,120.0,2.0,25.0,3.0,1.0,R1002P,5.0,125.0,"[38, 39, 40, 41, 42, 43, 44, 65, 66, 67, 68, 6..."
4,FR1,0.0,0.0,86.0,30.0,22.0,26.0,0.0,R1003P,56.0,142.0,"[38, 39, 40, 41, 42, 44, 46, 48, 69, 70, 71, 7..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,FR1,0.0,0.0,45.0,6.0,11.0,2.0,1.0,R1066P,8.0,53.0,"[38, 39, 41, 42, 63, 64, 65, 66, 67, 68, 69, 9..."
96,FR1,0.0,0.0,102.0,13.0,24.0,14.0,2.0,R1066P,27.0,129.0,"[39, 40, 41, 42, 43, 44, 45, 46, 67, 68, 69, 7..."
97,FR1,0.0,0.0,109.0,11.0,24.0,8.0,3.0,R1066P,19.0,128.0,"[39, 40, 41, 42, 43, 44, 66, 67, 69, 70, 92, 9..."
98,catFR1,0.0,0.0,99.0,22.0,25.0,13.0,0.0,R1066P,35.0,134.0,"[39, 40, 41, 43, 44, 45, 46, 69, 70, 72, 73, 7..."


In [17]:
# master[master['subject']=='R1001P']['Phase III Implant Date'].item()
master.columns

Index(['Subject Number', 'Event Name', 'Study site', 'Gender',
       'Years of Education', 'Age of seizure onset',
       'Age at Implant (calculated)', 'Phase II Implant Date',
       'Phase III Implant Date ', 'Traumatic Brain Injury (TBI) Notes',
       'Date of Pre-operative Report', 'BAI', 'BDI', 'FSIQ', 'GAI', 'VCI',
       'PRI', 'Pre-operative Report',
       'Prior Neurosurgical Procedures (can select more than one)',
       'Prior Surgery: Resection Detail', 'Prior Surgery: Callosotomy Detail',
       'Prior Surgery: Laser Ablation Detail', 'subject'],
      dtype='object')

In [19]:
#BEGINNING OF SESSION LEVEL ANALYSES
df_master = pd.DataFrame([])
#Create master dataframe of important session stats, demographics, and neuropsych.
for i in trial_df.index:
    try:
        sub = trial_df.iloc[i]['subject']
        eli = trial_df.iloc[i]['number of extra-list intrusions']
        pli = trial_df.iloc[i]['number of prior-list intrusions']
        correc = trial_df.iloc[i]['number of correct recalls']
        allrec = trial_df.iloc[i]['total number of trials left']
        lists = trial_df.iloc[i]['number of lists']
        exp = trial_df.iloc[i]['experiment']
        PLI_avg = float(pli/lists) #average per list
        ELI_avg = float(eli/lists)
        correc_avg = float(correc/lists)
        if allrec > 0: #remove sessions with no valid recalls and no BAI or BDI
            df_master = df_master.append({
                'subject':sub,
                'experiment':exp,
                'correct recalls':correc,
                'ELIs':eli,
                'PLIs':pli,
                'total ints': eli+pli,
                'lists':lists,
                'all recalls':allrec,
                'average correct recalls':correc_avg,
                'average ELI':ELI_avg,
                'average PLI':PLI_avg,
                'average total ints':ELI_avg+PLI_avg,
                'BAI':master[master['subject']==sub]['BAI'].item(),
                'BDI':master[master['subject']==sub]['BDI'].item(),
                'site':master[master['subject']==sub]['Study site'].item(),
                'gender':master[master['subject']==sub]['Gender'].item(),
                'years of education':master[master['subject']==sub]['Years of Education'].item(),
                'age of seizure onset':master[master['subject']==sub]['Age of seizure onset'].item(),
                'age at implant':master[master['subject']==sub]['Age at Implant (calculated)'].item(),
                'implant date':master[master['subject']==sub]['Phase II Implant Date'].item(),
                'reimplant date':master[master['subject']==sub]['Phase III Implant Date '].item(),
                'TBI notes':master[master['subject']==sub]['Traumatic Brain Injury (TBI) Notes'].item(),
                'date of neuropsych report':master[master['subject']==sub]['Date of Pre-operative Report'].item(),
                'FSIQ':master[master['subject']==sub]['FSIQ'].item(),
                'GAI':master[master['subject']==sub]['GAI'].item(),
                'VCI':master[master['subject']==sub]['VCI'].item(),
                'PRI':master[master['subject']==sub]['PRI'].item(),
                'prior neurosurgical procedures':master[master['subject']==sub]['Prior Neurosurgical Procedures (can select more than one)'].item()    
            }, ignore_index = True)
        else:
            print('removed row =', i, sub, exp, 'due to no valid recalls')
    except:
        print('failed for row =', i, sub, exp)
df_master = df_master.dropna(subset=['BAI', 'BDI'], how='all') #drop sessions from subjects that have neither score
df_master[:50]

/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


removed row = 12 R1015J FR1 due to no valid recalls
removed row = 158 R1100D FR1 due to no valid recalls
removed row = 159 R1100D FR1 due to no valid recalls


/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


,BAI,BDI,ELIs,FSIQ,GAI,PLIs,PRI,TBI notes,VCI,age at implant,...,experiment,gender,implant date,lists,prior neurosurgical procedures,reimplant date,site,subject,total ints,years of education
0,8.0,2.0,21.0,72.0,76.0,25.0,75,NaN,81,48.105026,...,FR1,Female,2014-10-08,22.0,NaN,NaT,University of Pennsylvania,R1001P,46.0,13.0
1,8.0,2.0,19.0,72.0,76.0,17.0,75,NaN,81,48.105026,...,FR1,Female,2014-10-08,23.0,NaN,NaT,University of Pennsylvania,R1001P,36.0,13.0
2,9.0,0.0,3.0,86.0,84.0,8.0,84,NaN,89,49.383743,...,FR1,Female,2014-11-12,25.0,NaN,NaT,University of Pennsylvania,R1002P,11.0,16.0
3,9.0,0.0,2.0,86.0,84.0,3.0,84,NaN,89,49.383743,...,FR1,Female,2014-11-12,25.0,NaN,NaT,University of Pennsylvania,R1002P,5.0,16.0
7,7.0,7.0,9.0,83.0,89.0,21.0,90,NaN,91,20.553582,...,FR1,Female,2015-01-28,25.0,NaN,NaT,University of Pennsylvania,R1006P,30.0,14.0
8,7.0,7.0,7.0,83.0,89.0,3.0,90,NaN,91,20.553582,...,FR1,Female,2015-01-28,13.0,NaN,NaT,University of Pennsylvania,R1006P,10.0,14.0
9,2.0,1.0,2.0,NaN,92.0,12.0,86,NaN,98,30.801454,...,FR1,Female,2014-12-02,23.0,NaN,NaT,Jefferson Hospital,R1010J,14.0,16.0
10,2.0,1.0,4.0,NaN,92.0,6.0,86,NaN,98,30.801454,...,FR1,Female,2014-12-02,23.0,NaN,NaT,Jefferson Hospital,R1010J,10.0,16.0
11,2.0,12.0,0.0,98.0,101.0,4.0,102,NaN,100,36.581290,...,catFR1,Female,2014-12-15,20.0,Resection,2015-02-18,Emory University,R1013E,4.0,16.0
12,7.0,15.0,1.0,NaN,89.0,3.0,92,NaN,89,54.509105,...,catFR1,Female,2015-01-06,9.0,Resection,NaT,Jefferson Hospital,R1015J,4.0,12.0
